# Trabajo Práctico: Agente para automatizar la búsqueda de repuestos

- **Curso:** DUIA - 2025, Módulo 6
- **Integrantes:** David Burckhardt, Martin Vazquez Arispe, Martin Caballero.
- **Objetivo:** Implementar un sistema inteligente que automatice la búsqueda, ranking y pedido de repuestos para una empresa distribuidora.

---
##  Índice del Notebook

1. **Consigna del trabajo**
2. **Configuracion de API Keys**

---

## 1. Consigna: agente(s) para automatizar la búsqueda de repuestos
- Dada una solicitud de repuestos específicos para una empresa distribuidora, un
agente debe identificar las especificaciones de dichos repuestos (según un
catálogo), a fin de poder buscarlos.
- El agente busca en primer lugar en el inventario de la empresa, y en en caso de
no encontrarlos (puede ser que encuentre solo algunos de ellos), debe consultar
en catálogos de proveedores.
- El sistema extrae información de las opciones encontradas, y genera un ranking
de alternativas, priorizando: 
    - Repuestos internos (si están disponibles).
    - Proveedores externos según criterios de optimización (por ej. costo-beneficio).
- Para repuestos internos: 
    - Se genera una orden de retiro del inventario y se notifica al almacén para su preparación. 
- Para repuestos externos: 
    - se envía un email automatizado al proveedor seleccionado para formalizar el pedido.

- Finalmente, se agenda la fecha estimada de entrega y detalles del pedido en el
sistema de seguimiento.
- Pueden incluirse pasos de "human in the loop" para verificar resultados antes de
tomar acciones

## 2. Dependencias necesarias

In [1]:
from langgraph.graph import StateGraph, START, END
from langchain_core.messages import HumanMessage, AIMessage
from langgraph.graph.message import add_messages
from typing import TypedDict, Annotated, Optional, Dict, List
from pydantic import BaseModel, Field
from langgraph.checkpoint.memory import MemorySaver
import re
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.output_parsers import PydanticOutputParser
import json
from pymongo import MongoClient
import os
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq



/opt/anaconda3/envs/DUIA/lib/python3.14/site-packages/langchain_core/_api/deprecation.py:26: UserWarning: Core Pydantic V1 functionality isn't compatible with Python 3.14 or greater.
  from pydantic.v1.fields import FieldInfo as FieldInfoV1


## 3. Configuración de API Keys y variables de entorno

Cargamos la `GROQ_API_KEY` desde el archivo `.env` e inicializamos el cliente LLM.

**Nota:** Asegúrate de tener un archivo `.env` en el directorio raíz con:
```
GROQ_API_KEY=tu_clave_aqui
```

In [2]:
load_dotenv()

# Verificar que la API key está configurada
api_key = os.getenv("GROQ_API_KEY")
if not api_key:
    raise ValueError("GROQ_API_KEY no encontrada en .env")

# Inicializar el LLM de Groq
llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    temperature=0.1,
    api_key=api_key
)

print("LLM de Groq inicializado correctamente")
print(f"   Modelo: {llm.model_name}")
print(f"   Temperature: {llm.temperature}")


LLM de Groq inicializado correctamente
   Modelo: llama-3.3-70b-versatile
   Temperature: 0.1


## 4. Conexión a Mongo DB

Cargamos la `MONGO_URI` desde el archivo `.env` e inicializamos.
IMPORTANTE: Contar con un usuario creado en la base de datos de Mongo. 

**Nota:** Asegúrate de tener un archivo `.env` en el directorio raíz con:
```
MONGO_URI=tu_clave_aqui
```

In [3]:
# Conectar a MongoDB Atlas
MONGO_URI = os.getenv("MONGO_URI")

if not MONGO_URI:
    raise ValueError("MONGO_URI no encontrada en .env")

client = MongoClient(MONGO_URI)
db = client.db_respuestos
collection = db.repuestos

print("✅ Conexión a MongoDB establecida")
print(f"   Base de datos: db_respuestos")
print(f"   Colección: repuestos")

✅ Conexión a MongoDB establecida
   Base de datos: db_respuestos
   Colección: repuestos


## 5. Definicion del Estado 

In [4]:
class ValidationRequest(BaseModel):
    is_parts_request: bool = Field(
        default=False,
        description="True si la consulta es una solicitud de repuestos o piezas. False si es una pregunta general o spam."
    )
    message: str = Field(
        default="",
        description="Mensaje del agente. Si es un pedido de repuestos, debe incluir los pasos siguientes. Indicar al cliente que debe se deben realizar consultas sobre repuestos."
    )

#Definimos el esquema mejorado
class AgentState(TypedDict):
    validation_result: ValidationRequest
    messages: Annotated[list, add_messages]
    codigos_repuestos: Optional[str]  # Código del repuesto (R-XXXX)
    info_completa: bool  # Si tenemos toda la información necesaria

    # Resultados de la búsqueda interna
    resultados_internos: Optional[Dict[str, List[Dict]]]  

    # Resultados de la búsqueda externa
    resultados_externos: Optional[Dict[str, List[Dict]]]

    # Análisis de disponibilidad por código
    disponibilidad: Optional[Dict[str, str]]  # {"R-0001": "full", "R-0002": "none"}
    codigos_para_externos: Optional[List[str]]

    # Reranking
    recomendaciones_llm: Optional[str]

## 6. Definicion de todas las Chains del LLM

### Chain de Conversación Inicial

In [5]:
with open('prompts/chat_inicial_prompt.txt', 'r', encoding='utf-8') as f:
    SYSTEM_PROMPT = f.read()

prompt = ChatPromptTemplate.from_messages([
    ("system", SYSTEM_PROMPT),
    ("placeholder", "{messages}")
])

conversational_chain = prompt | llm

### Chain de Validación de Intencion

In [6]:
with open('prompts/intention_classifier_prompt.txt', 'r', encoding='utf-8') as f:
    CLASSIFIER_SYSTEM_PROMPT = f.read()

validation_prompt = ChatPromptTemplate.from_messages([
    ("system", CLASSIFIER_SYSTEM_PROMPT),
    ("placeholder", "{messages}")
])

parser = PydanticOutputParser(pydantic_object=ValidationRequest)

validation_prompt_con_instrucciones = ChatPromptTemplate.from_messages(
    validation_prompt.messages + [
        ("human", "{format_instructions}")
    ]
).partial(
    format_instructions=parser.get_format_instructions()
)

validation_chain = (
    validation_prompt_con_instrucciones 
    | llm 
    | parser
)

### Chain de Reranking

In [7]:
with open('prompts/reranking_prompt.txt', 'r', encoding='utf-8') as f:
    RERANKING_PROMPT = f.read()

ranking_prompt_template = ChatPromptTemplate.from_messages([
    ("system", RERANKING_PROMPT),
    ("user", "Analiza las siguientes opciones y genera un ranking completo:\n\n{opciones_texto}")
])

ranking_chain = ranking_prompt_template | llm

## 7. Definición de los Nodos del Grafo

In [ ]:
def classify_request(state: AgentState) -> AgentState:
    """
        Clasifica la solicitud del usuario.
    """
    messages: list[BaseMessage] = state['messages']
    validation_result_object = validation_chain.invoke({"messages": messages})
    
    return {"validation_result": validation_result_object}

def set_val_message(state: AgentState) -> AgentState:
    """
        Establece el mensaje de validación.
    """
    return {"messages": [state['validation_result'].message]}

def route_classification(state: AgentState) -> str:
    """
        Ruta de clasificación.
    """
    if state['validation_result'].is_parts_request:
        return "continue"
    else:
        return "end"

def identificar_repuestos(state: AgentState) -> AgentState:
    """
        Identifica los códigos de repuesto en los mensajes del usuario.
    """
    messages = state["messages"]
    response = conversational_chain.invoke({"messages": messages})
    
    # Extraer código de repuesto si está presente en los mensajes
    codigos_repuestos = state.get("codigos_repuestos")
    codigos_encontrados = set()
    
    # Buscar patrón R-XXXX en todos los mensajes del usuario
    for msg in messages:
        if isinstance(msg, HumanMessage):
            # Buscar patrón R-XXXX (ej: R-0001, R-0123)
            matches = re.findall(r'R-\d{4}', msg.content, re.IGNORECASE)
            for match in matches:
                codigos_encontrados.add(match.upper())
    
    # Determinar si tenemos información completa
    codigos_list = sorted(list(codigos_encontrados))
    info_completa = len(codigos_list) > 0
    
    return {
        "messages": [response],
        "codigos_repuestos": codigos_list,
        "info_completa": info_completa
    }

#Defino la funcion para decidir si continuamos o terminamos
def tiene_info_suficiente(state: AgentState) -> str:
    # Si tenemos el código del repuesto, terminamos
    if not(state.get("info_completa", False) and state.get("codigos_repuestos")):
        return "request_more_info"
    else:
        print(f"\n✅ Código de repuesto identificado: {state['codigos_repuestos']}")
        return "continue"

def search_internal_parts(state: AgentState) -> AgentState:
    """
        Busca el repuesto en MongoDB usando el código identificado.
        Muestra los resultados por pantalla de forma detallada.
    """
    codigos = state.get("codigos_repuestos")
    try:
        # Buscar en MongoDB por id_repuesto
        resultados = list(collection.find({
            "id_repuesto": {"$in": codigos},
            "proveedor_tipo": "INTERNAL"
        }))

        resultados_por_codigo = {codigo: [] for codigo in codigos}
        for resultado in resultados:
            codigo = resultado.get("id_repuesto")
            if codigo in resultados_por_codigo:
                # Limpiar _id de MongoDB para serialización
                if '_id' in resultado:
                    resultado['_id'] = str(resultado['_id'])
                resultados_por_codigo[codigo].append(resultado)
        
                # Mostrar resultados básicos
        for codigo in codigos:
            opciones = resultados_por_codigo[codigo]
            if opciones:
                print(f"✅ {codigo}: {len(opciones)} proveedor(es) interno(s) encontrado(s)")
            else:
                print(f"❌ {codigo}: No encontrado en inventario interno")
        
        print(f"{'='*80}\n")
        
        return {"resultados_internos": resultados_por_codigo}

    except Exception as e:
        print(f"❌ Error al buscar en la base de datos: {e}")
        return {"resultados_internos": {codigo: [] for codigo in codigos}}

def check_stock(state: AgentState) -> AgentState:
    """
        Verifica la disponibilidad de los repuestos en el inventario interno.
        Muestra los resultados por pantalla de forma detallada.
    """
    codigos = state.get("codigos_repuestos")
    resultados_internos = state.get("resultados_internos", {})

    disponibilidad = {}
    necesita_externos = []

    for codigo in codigos:
        opciones_internas = resultados_internos.get(codigo, [])
        if not opciones_internas:
            disponibilidad[codigo] = "none"
            necesita_externos.append(codigo)
        else:
            total_stock = sum(opcion.get("stock_disponible", 0) for opcion in opciones_internas)
            if total_stock > 0:
                # Hay stock disponible
                disponibilidad[codigo] = "available_internal"
                print(f"✅ {codigo}: Disponible en inventario interno ({total_stock} unidades)")
                
                # Mostrar mejores opciones
                mejor_opcion = max(opciones_internas, key=lambda x: x.get("stock_disponible", 0))
                print(f"   └─ Mejor opción: {mejor_opcion.get('proveedor_nombre')} "
                      f"({mejor_opcion.get('stock_disponible')} unidades, "
                      f"{mejor_opcion.get('lead_time_dias')} días)")
            else:
                # Encontrado pero sin stock
                disponibilidad[codigo] = "no_stock_internal"
                necesita_externos.append(codigo)
                print(f"⚠️  {codigo}: Encontrado pero sin stock suficiente en el inventario interno")

    return {
        "disponibilidad": disponibilidad,
        "codigos_para_externos": necesita_externos  # ← Nueva clave en el estado
    }

def search_external_parts(state: AgentState) -> AgentState:
    """
    Busca en proveedores externos solo los repuestos que lo necesitan.
    """
    codigos_para_externos = state.get("codigos_para_externos", [])
    print(f"Buscando {len(codigos_para_externos)} repuesto(s): {', '.join(codigos_para_externos)}\n")
    
    try:
        # Búsqueda en batch
        resultados = list(collection.find({
            "id_repuesto": {"$in": codigos_para_externos},
            "proveedor_tipo": "EXTERNAL"
        }))
        
        # Organizar por código
        resultados_por_codigo = {codigo: [] for codigo in state.get("codigos_repuestos", [])}
        for resultado in resultados:
            codigo = resultado.get("id_repuesto")
            if codigo in resultados_por_codigo:
                if '_id' in resultado:
                    resultado['_id'] = str(resultado['_id'])
                resultados_por_codigo[codigo].append(resultado)
                print(f"✅ {codigo}: {len(resultados_por_codigo[codigo])} proveedor(es) externo(s) encontrado(s)")
        
        return {"resultados_externos": resultados_por_codigo}
        
    except Exception as e:
        print(f"❌ Error al buscar en proveedores externos: {str(e)}\n")
        return {"resultados_externos": {}}

def need_external_parts(state: AgentState) -> str:
    """
    Decide si se necesita buscar en proveedores externos.
    """
    codigos_para_externos = state.get("codigos_para_externos", [])
    
    if codigos_para_externos:
        return "search_external"
    else:
        return "reranking"

def generate_ranking(state: AgentState) -> AgentState:
    from utils import format_options_for_llm
    """
    Genera ranking usando SOLO el LLM.
    """
    codigos = state.get("codigos_repuestos", [])
    resultados_internos = state.get("resultados_internos", {})
    resultados_externos = state.get("resultados_externos", {})
    
    print(f"\n{'='*80}")
    print(f"🏆 GENERANDO RANKING")
    print(f"{'='*80}\n")
    
    # Preparar información para el LLM
    opciones_para_llm = []
    
    for codigo in codigos:
        internos = resultados_internos.get(codigo, [])
        externos = resultados_externos.get(codigo, [])
        
        # Combinar todas las opciones SIN ordenar ni scoring
        todas_opciones = []
        
        for opcion in internos:
            todas_opciones.append({**opcion, "tipo": "INTERNO"})
        for opcion in externos:
            todas_opciones.append({**opcion, "tipo": "EXTERNO"})
        if todas_opciones:
            # Formatear para el LLM
            opciones_para_llm.append(
                format_options_for_llm(codigo, todas_opciones)
            )
    
    if not opciones_para_llm:
        print("⚠️ No hay opciones para rankear\n")
        return {
            "ranking_por_codigo": {},
            "recomendaciones_llm": "No hay opciones disponibles."
        }
    
    # Crear el texto completo para el LLM
    opciones_texto = "\n\n".join(opciones_para_llm)
    
    try:
        # Invocar el LLM para que haga el ranking
        recomendaciones = ranking_chain.invoke({"opciones_texto": opciones_texto})
        recomendaciones_texto = recomendaciones.content
        print(recomendaciones_texto)
        
    except Exception as e:
        print(f"❌ Error al obtener ranking del LLM: {e}\n")
        recomendaciones_texto = "Error al generar ranking automático."
    
    return {
        "recomendaciones_llm": recomendaciones_texto
    }

## 8. Generación del Grafo

In [9]:
memory = MemorySaver()

#Defino el grafo
graph_builder = StateGraph(AgentState)

#Agrego el nodo al grafo
graph_builder.add_node("identificar_repuestos", identificar_repuestos)
graph_builder.add_node("validation", classify_request)
graph_builder.add_node("set_val_message", set_val_message)
graph_builder.add_node("search_internal_parts", search_internal_parts)
graph_builder.add_node("check_stock", check_stock)
graph_builder.add_node("need_external_parts", need_external_parts)
graph_builder.add_node("search_external_parts", search_external_parts)
graph_builder.add_node("reranking", generate_ranking)

#Conecto los nodos (Solo uno por ahora)
graph_builder.add_edge(START, "validation")
graph_builder.add_conditional_edges(
    "validation",
    route_classification,
    {
        "continue": "identificar_repuestos",
        "end": "set_val_message"
    }
)
graph_builder.add_edge("set_val_message", END)
graph_builder.add_conditional_edges(
    "identificar_repuestos",
    tiene_info_suficiente,
    {
        "continue": "search_internal_parts",  
        "request_more_info": "validation",
        "end": END
    }
)
graph_builder.add_edge("search_internal_parts", "check_stock")
graph_builder.add_conditional_edges(
    "check_stock",
    need_external_parts,
    {
        "search_external": "search_external_parts",
        "reranking": "reranking"
    }
)
graph_builder.add_edge("search_external_parts", "reranking")
graph_builder.add_edge("reranking", END)
graph = graph_builder.compile(
    checkpointer=memory, 
    interrupt_after=["identificar_repuestos"]
)

## 9. Inicialización del agente

In [10]:

def iniciar_agente(mensaje_usuario: str):
    config = {"configurable": {"thread_id": "1"}}
    
    # Estado inicial con todos los campos
    estado_inicial = {
        "messages": [HumanMessage(mensaje_usuario)],
        "validation_result": None,
        "codigos_repuestos": None,
        "info_completa": False,
        "resultados_internos": {},
        "resultados_externos": {},
        "disponibilidad": None,
        "codigos_para_externos": None,
        "recomendaciones_llm": None,
        "decision_usuario": None,  # NUEVO
        "opcion_seleccionada": None  # NUEVO
    }
    
    result = graph.invoke(estado_inicial, config)
    
    # Loop de conversación
    while True:
        print(f"\n🤖 Agente: {result['messages'][-1].content}")
        
        # Verificar si identificamos codigos de repuestos
        if result.get("info_completa", False):
            print(f"\n{'='*60}")
            print(f"📋 Información recopilada:")
            print(f"   Código del repuesto: {result['codigos_repuestos']}")
            print(f"{'='*60}")
            result = graph.invoke(None, config)
            print("El grafo termino de ejecutar")
            break
        
        # Si no identificamos codigos de repuestos, pedir nuevo mensaje al usuario
        nuevo_mensaje = input("\n👤 Tú: ")
        
        if nuevo_mensaje.lower() in ["salir", "exit", "quit"]:
            print("\n👋 Conversación terminada")
            break
        
        # Volvemos a ejecutar el grafo con el nuevo mensaje
        nuevo_estado = {"messages": [HumanMessage(content=nuevo_mensaje)]}
        result = graph.invoke(nuevo_estado, config)
    
    return result

## 10. Inicio de Conversación

### Ejemplos de uso del agente

**Ejemplos de códigos de repuestos en la base de datos:**
- R-0001: Rodamiento rígido de bolas 6204 2RS
- R-0002: Filtro de aceite motor diésel
- R-0005: Bomba centrífuga 3 HP
- R-0010: Manómetro glicerina 0-16 bar

**Categorías disponibles:**
- RODAMIENTO, FILTRO, CORREA, SENSOR, BOMBA, ELECTRICO, NEUMATICA, MECANICO, INSTRUMENTO


In [11]:
#main
print("="*60)
print("🔧 SISTEMA DE BÚSQUEDA DE REPUESTOS")
print("="*60)
print("\nBienvenido al sistema de búsqueda de repuestos.")
print("El agente te ayudará a encontrar el repuesto que necesitas.")
print("\nPuedes escribir 'salir' en cualquier momento para terminar.\n")
print("-"*60)

mensaje_usuario = input("\n👤 Tú: ")

#Para debugging
#mensaje_usuario = "Repuesto con codigo R-0005 y R-0002"

resultado = iniciar_agente(mensaje_usuario)

#Repuesto con el codigo R-0001
#Repuesto con codigo R-0005 y R-0002


🔧 SISTEMA DE BÚSQUEDA DE REPUESTOS

Bienvenido al sistema de búsqueda de repuestos.
El agente te ayudará a encontrar el repuesto que necesitas.

Puedes escribir 'salir' en cualquier momento para terminar.

------------------------------------------------------------

🤖 Agente: Tu mensaje parece ser un saludo. ¿Qué repuesto necesitas buscar?

👋 Conversación terminada
